In [15]:
import pandas as pd
f1 = pd.read_csv("emails.csv")
f2 = pd.read_csv("spam_ham_dataset.csv")

data= pd.concat([pd.DataFrame(f1["text"], columns=["text"]), pd.DataFrame(f2["text"], columns=["text"])])

label= pd.concat([pd.DataFrame(f1["label"], columns=["label"]), pd.DataFrame(f2["label"], columns=["label"])])
print(label)

      label
0         1
1         1
2         1
3         1
4         1
...     ...
5166      0
5167      0
5168      0
5169      0
5170      1

[10899 rows x 1 columns]
